# Calculation of excited state energies

- Many algorithms are proposed and used for classical methods
 - Monte-Carlo (Variational, Diffusion)
 - Symmetry/orthogonality constraints on variational loss[Prof. Izmaylov's lecture]
 - etc...

- Several algorithms are proposed for VQE
 - Enforce orthogonality in input (SSVQE [2](https://arxiv.org/pdf/1810.09434.pdf), [3](https://dojo.qulacs.org/ja/latest/notebooks/6.3_subspace_search_VQE.html))
 - Enforce orthogonality in output
 - etc...

## SSVQE Algorithm
1. Select target exited state. Let it be $k$-th state for now.
2. Minimize $\mathcal{L}_{1}(\boldsymbol{\theta})=\sum_{j=0}^{k}\left\langle\varphi_{j}\left|U^{\dagger}(\boldsymbol{\theta}) H U(\boldsymbol{\theta})\right| \varphi_{j}\right\rangle$, where $\left\{\left|\varphi_{j}\right\rangle\right\}_{j=0}^{k}$ are k different computational basis are k different computational basis
3. Maximize $\mathcal{L}_{2}(\phi)=\left\langle\varphi_{k}\left|V^{\dagger}(\phi) U^{\dagger}\left(\boldsymbol{\theta}^{*}\right) H U\left(\boldsymbol{\theta}^{*}\right) V(\boldsymbol{\phi})\right| \varphi_{k}\right\rangle$

This works because
1. If we minimize $\mathcal{L}_{1}(\boldsymbol{\theta})$, it should be $\sum_{j=0}^{k} E_k$, therefore the space after applying unitary $U(\boldsymbol{\theta})$ only includes $\left\{\left|E_{j}\right\rangle\right\}_{j=0}^{k}$
2. If we maximize $\mathcal{L}_{2}(\phi)$, then we are searching maximum in the space where the largest enegy is $E_k$, eventually outputs $E_k$

## SSVQE algorithm variant with single loss is

1. Minimize $ \mathcal{L}_{w}(\boldsymbol{\theta})=w\left\langle\varphi_{k}\left|U^{\dagger}(\boldsymbol{\theta}) H U(\boldsymbol{\theta})\right| \varphi_{k}\right\rangle+ \sum_{j=0}^{k-1}\left\langle\varphi_{j}\left|U^{\dagger}(\boldsymbol{\theta}) H U(\boldsymbol{\theta})\right| \varphi_{j}\right\rangle $ where  $w \in (0, 1)$
2. Calculate expectation value w.r.t. $\left|\varphi_{k}\right\rangle$

This works because

1. $k$ th state has looser regularization compared to others because of $w$, therefore it optimizes $j\ne k $ first to map $\left|\varphi_{j}\right\rangle $ to $\left\{\left|E_{j}\right\rangle\right\}_{j=0}^{j=k-1}$, while maping $\left|\varphi_{k}\right\rangle$ to $\left\{\left|E_{k}\right\rangle\right\}$


### Here we show SSVQE result for H2
- There seems to be a stagnation point around -0.58 [eV] for 1st excited state. It is also shown in the [author's tutorial](https://dojo.qulacs.org/ja/latest/notebooks/6.3_subspace_search_VQE.html)
- We cannot exceed this point in this calculation

In [45]:
import numpy as np
import tequila as tq
from tequila import QCircuit
from tequila.circuit.gates import X, Ry, Rz, CZ
from utility import *
threshold = 1e-6 #Cutoff for UCC MP2 amplitudes and QCC ranking gradients
h2, _ = get_qubit_hamiltonian(mol='h2', geometry=0.977, basis='sto3g', qubit_transf='jw')
mat = QubitHamiltonian.from_openfermion(h2).to_matrix()
eigval, eigvec = np.linalg.eigh(mat)

print('ground state energy', eigval[0])
print('1st excited state energy', eigval[1])

ground state energy -1.105933352304692
1st excited state energy -0.7329846745974556


In [51]:
trotter_steps = 1
xyz_data = get_molecular_data('h2', geometry=0.977, xyz_format=True)
h2 = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set='sto-3g')
H = h2.make_hamiltonian()


def he_ansatz_circuit(n_qubit, depth, theta_list):
    """he_ansatz_circuit
    Returns hardware efficient ansatz circuit.

    Args:
        n_qubit (:class:`int`):
            the number of qubit used (equivalent to the number of fermionic modes)
        depth (:class:`int`):
            depth of the circuit.
        theta_list (:class:`numpy.ndarray`):
            rotation angles.
    Returns:
        :class:`qulacs.QuantumCircuit`
    """
    circuit = QCircuit()
    circuit+=Ry(target=0, angle=tq.Variable('a'))
    circuit+=Ry(target=0, angle=tq.Variable('b'))
    for d in range(depth):
        for i in range(n_qubit):
            circuit += Ry(target=i, angle=tq.Variable(f'{d}-{i}-y'))
            circuit += Rz(target=i, angle=tq.Variable(f'{d}-{i}-z'))
        for i in range(n_qubit//2):
            circuit += CZ(2*i, 2*i+1)
        for i in range(n_qubit//2-1):
            circuit += CZ(2*i+1, 2*i+2)
    for i in range(n_qubit):
        circuit += Ry(target=i, angle=tq.Variable(f'{i}-y-2'))
        circuit += Rz(target=i, angle=tq.Variable(f'{i}-z-2'))

    return circuit

In [52]:
n_qubit = 2* h2.n_orbitals
U_UCCSD = h2.make_uccsd_ansatz(initial_amplitudes='mp2',threshold=threshold, trotter_steps=trotter_steps)
# U_UCCSD2 = h2.make_uccsd_ansatz(initial_amplitudes='mp2',threshold=threshold, trotter_steps=trotter_steps).insert_gates([0], [X(0)])
depth =n_qubit
init_theta_list = np.random.random(2*n_qubit*(depth+1)+2)*1e-1
U = he_ansatz_circuit(n_qubit, n_qubit, init_theta_list)
U2 = he_ansatz_circuit(n_qubit, n_qubit, init_theta_list).insert_gates([0], [X(0)])

In [55]:
E = tq.ExpectationValue(H=H, U=U) + 0.4 *  tq.ExpectationValue(H=H, U=U2)

print('\nStarting optimization:\n')

result = tq.minimize(objective=E, method="BFGS", initial_values={k: np.random.random(1) * 1e-1 for k in E.extract_variables()}, tol=1e-6, maxiter=200, disp=True)



Starting optimization:

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 2 expectationvalues
gradient        : 252 expectationvalues

active variables : 42

E=+0.24536128  angles= {a: 0.04538922377411515, b: 0.05370265947744696, 0-0-y: 0.0023481091491714292, 0-0-z: 0.037650579616994174, 0-1-y: 0.049770408935881895, 0-1-z: 0.08552800841709084, 0-2-y: 0.007371683539382368, 0-2-z: 0.017552775937705746, 0-3-y: 0.0793405502218647, 0-3-z: 0.0670829624459799, 1-0-y: 0.012277448503651846, 1-0-z: 0.021508467525535202, 1-1-y: 0.08940549020133477, 1-1-z: 0.06348656214543978, 1-2-y: 0.05460028561584192, 1-2-z: 0.03926547490787039, 1-3-y: 0.09195635219227799, 1-3-z: 0.003669322527655172, 2-0-y: 0.09020347150418362, 2-0-z: 0.034565882472223954, 2-1-y: 0.030053949822394732, 2-1-z: 0.039091733786366635, 2-2-y: 0.08319731367210459, 2-2-z: 0.0239

Convergence to -1.39912450 gives $E_1$ as -1.39912450 - (- 1.1056) / 0.4 $\sim$ -0.733, which is fairly good.